In [ ]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service import catalog, jobs, pipelines

w = WorkspaceClient()

# ======================
# Setup
# ======================

# The following function simplifies the replication of multiple tables from the same schema
def replicate_tables_from_db_schema(db_catalog_name, db_schema_name, db_table_names):
  return [pipelines.IngestionConfig(
            table = pipelines.TableSpec( 
            source_catalog=db_catalog_name,
            source_schema=db_schema_name,
            source_table=table_name,
            destination_catalog=target_catalog_name,
            destination_schema=target_schema_name,
          )) for table_name in db_table_names]

# The following function simplifies the replication of an entire DB schemas
def replicate_full_db_schema(db_catalog_name, db_schema_names):
  return [pipelines.IngestionConfig(
            schema = pipelines.SchemaSpec( 
            source_catalog=db_catalog_name,
            source_schema=db_schema_name,
            destination_catalog=target_catalog_name,
            destination_schema=target_schema_name,
          )) for db_schema_name in db_schema_names]

# Cluster configuration for the Gateway pipeline
gateway_cluster_spec = pipelines.PipelineCluster(
    driver_node_type_id = "Standard_E8d_v4",
    node_type_id = "Standard_D3_v2",  
)


# ======================
# Variables
# ======================

# The name of the UC connection with the credentials to access the source database
connection_name = "sql-server"

# The name of the UC catalog and schema to store the replicated tables
target_catalog_name = "jpg_ws_us_3"
target_schema_name = "default"

# The name of the UC catalog and schema to store the staging volume with intermediate
# CDC and snapshot data.
# Use the destination catalog/schema by default
stg_catalog_name = target_catalog_name 
stg_schema_name = target_schema_name 

# The name of the Gateway pipeline to create
gateway_pipeline_name = "cdc_gateway"

# The name of the Ingestion pipeline to create
ingestion_pipeline_name = "cdc_ingestion"

# The name of the SQL Server catalog to replciate from
sql_server_catalog = ""

# List of SQL Server schemas to replicate
sql_server_schemas = [""]

# Construct the full list of tables to replicate
# IMPORTANT: The letter case of the catalog, schema and table names MUST MATCH EXACTLY the case used in the source database system tables
tables_to_replicate = replicate_full_db_schema(sql_server_catalog, sql_server_schemas)
# Append tables from additional schemas as needed
#  + replicate_tables_from_db_schema("REPLACE_WITH_DBNAME", "REPLACE_WITH_SCHEMA_NAME_2", ["table3", "table4"])

# Customize who gets notified about failures
notifications = [
  pipelines.Notifications(
      email_recipients = [ w.current_user.me().user_name ],
      alerts = [ "on-update-failure", "on-update-fatal-failure", "on-flow-failure"]
      )
  ]


# determine the connection id
connection_id = w.connections.get(connection_name).connection_id

gateway_def = pipelines.IngestionGatewayPipelineDefinition(
      connection_name=connection_name,
      connection_id=connection_id,
      gateway_storage_catalog=stg_catalog_name, 
      gateway_storage_schema=stg_schema_name,
      gateway_storage_name = gateway_pipeline_name)

p = w.pipelines.create(
    name = gateway_pipeline_name, 
    gateway_definition=gateway_def, 
    notifications=notifications,
    clusters= [ gateway_cluster_spec ] if gateway_cluster_spec is not None else None
    )
gateway_pipeline_id = p.pipeline_id

print(f"Gateway pipeline {gateway_pipeline_name} created: {gateway_pipeline_id}")

ingestion_def = pipelines.IngestionPipelineDefinition(
    ingestion_gateway_id=gateway_pipeline_id,
    objects=tables_to_replicate,
    )
p = w.pipelines.create(
    name = ingestion_pipeline_name, 
    ingestion_definition=ingestion_def, 
    notifications=notifications,
    serverless=True,
    photon=True,
    continuous=False,
    )
ingestion_pipeline_id = p.pipeline_id

print(f"Ingestion pipeline {ingestion_pipeline_name} created: {ingestion_pipeline_id}")
